In [1]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import numpy as np


In [3]:
#archivo: "C:\Users\Usuario\Henry\PI01MLOPs\DataSet\Users_items\users_items.json.gz"

In [4]:
# Desarrollo función leer_Json_gzip() que lee los archivos 'comprimidos.json.gz` y devuelve una lista de diccionarios Python.
def leer_json_gzip(users_items):

    # Abrir el archivo users_items.json.gz en modo de lectura y decodificamos el contenido del archivo.
    with gzip.open(users_items, 'rt', encoding='utf-8') as miArchivo:

        # Recorremos el contenido del archivo y evaluamos cada línea como una expresión Python.
        # El resultado de la evaluación es un diccionario Python.
        # La función strip() elimina los espacios en blanco al principio y al final de la línea.
        return [ast.literal_eval(line.strip()) for line in miArchivo]
    #Usamos ast.literal_eval para convertir cada linea en un diccionario y lo agregamos a la lista

In [5]:
datos_users_items = []
for i  in gzip.open('users_items.json.gz'):
    datos_users_items.append(ast.literal_eval(i.decode('utf-8')))

df_users_items = pd.DataFrame(datos_users_items)


In [ ]:
df_users_items

In [ ]:
df_users_items.head()

In [8]:
df_users_items = df_users_items.dropna()

In [ ]:
df_users_items.info()

In [11]:
# borrar de una vez todas las columnas que no sirven
# user_id	items_count	steam_id	user_url items

In [ ]:
df_users_items

In [11]:
#borro colume user_url

df_users_items = df_users_items.drop('user_url', axis=1)


In [12]:
df_users_items.to_parquet('users_items.parquet', engine='pyarrow')

In [ ]:
df_users_items.head()

In [ ]:
df_users_items.info()

In [15]:
# Dado que el archivo esta muy largo no desanidaré la columna hasta que el enunciado no lo indique y queda asi el archivo que ya está manejable
# Al hacer la función veo que debo desanidar columna "items" para encontrar el tiempo jugado

df_users_items_exploded = df_users_items.explode('items')

In [16]:
# Normalizo los datos y creo un nuevo data frame desanidado
df_users_items_desanidado = pd.json_normalize(df_users_items_exploded['items'].dropna())

In [17]:
# Volver a indexar
df_users_items_desanidado.reset_index(inplace=True)
df_users_items_exploded.reset_index(inplace=True)

In [18]:
#  Concateno Data Frames y borro columna original items
df_users_items_concatenado = pd.concat([df_users_items_exploded, df_users_items_desanidado], axis=1)
df_users_items_concatenado =  df_users_items_concatenado.drop(columns=['items'])

In [ ]:
df_users_items_concatenado

In [ ]:
df_users_items_concatenado.info()

In [21]:
# al explodar item subio de 3M a 276, empezaré por borrar drpna
df_users_items_concatenado=df_users_items_concatenado.dropna()

In [ ]:
df_users_items_concatenado.info()

In [20]:
#Para disminuir espacio borro columnas que no usaré.
# Borro index	user_id	item_id	 playtime_2weeks
# Lista de columnas a borrar
#columnas_a_borrar = index	user_id	index	item_id	item_name	playtime_forever	playtime_2weeks
#Borrar una sola columna
# del df['B']


In [ ]:
df_users_items_concatenado.info()

In [24]:
df_users_items = df_users_items_concatenado

In [ ]:
df_users_items.info()

In [26]:
df_users_items = df_users_items.dropna()

In [ ]:
df_users_items.info()

In [28]:
df_users_items_concatenado = df_users_items_concatenado.drop('user_id', axis=1)

In [ ]:
df_users_items_concatenado.head()

In [ ]:
df_users_items_concatenado.info()

In [31]:
#reviso por si acaso los duplicdos
df_users_items_concatenado_sinduplicado = df_users_items_concatenado.drop_duplicates()


In [32]:
df_users_items_concatenado = df_users_items_concatenado_sinduplicado

In [ ]:
df_users_items_concatenado.info()

In [ ]:
df_users_items_concatenado.head()

In [35]:
df_users_items = df_users_items_concatenado

In [ ]:
df_users_items.info()

In [ ]:
df_users_items.head()

In [38]:
df_users_items = df_users_items.drop_duplicates()

In [ ]:
df_users_items.info()

In [ ]:
df_users_items.head()

In [41]:
df_users_items = df_users_items.drop(columns=['index'])


In [42]:
#Voy a borrar la fila que tenga 0 en playtimeforever

df_users_items = df_users_items.drop(df_users_items[(df_users_items['playtime_forever'] == 0)].index)

In [43]:
#voy a borrar la cuarte parte del dataframe por que la muestra es demasiado grande y no vale la pena
num_filas = len(df_users_items)
tercera_parte = int(num_filas / 1.8)
# Obtener índices aleatorios de la tercera parte de las filas
indices_a_borrar = np.random.choice(df_users_items.index, size=tercera_parte, replace=False)
df_users_items = df_users_items.drop(indices_a_borrar)



In [ ]:
df_users_items.info()

In [45]:
df_users_items.to_json('df_users_items.json', orient='records')
